# Practicing a Histogram Orientation Gradient (HOG)

In [1]:
import cv2
import open3d as o3d

%load_ext autoreload
%autoreload 2
%autosave 180

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Autosaving every 180 seconds


# Step 1: Preprocessing 

In [4]:
# Load the image
im = cv2.imread('/home/daniel-choate/ASAR/s2/TerrainNav/Match_Demos/bolt.png')

# Specify the desired dimensions
width_s, width_e = 70, 134  # desired width in pixels
height_s, height_e = 50, 178  # desired height in pixels
# new_dim = (width, height)

print('The selected pixel ratio is', width_e-width_s, 'x', height_e-height_s)

# Crop the image 
im_crop = im[height_s:height_e, width_s:width_e] #, height_s:height_e]

# # Or, RESIZE the image
# im_resize = cv2.resize(im, new_dim)

# Save the resized image
# cv2.imwrite('path/to/your/resized_image.png', resized_image)

# Optionally, display the resized image
# Can do with opencv, don't need open3d
cv2.imshow('Cropped Image', im_crop)
cv2.waitKey(0)
cv2.destroyAllWindows()


The selected pixel ratio is 64 x 128
